In [12]:
# run this if you want to run the example from the examples folder without installing the package
import sys
sys.path.insert(0, "../")

In [13]:
# import the package
import osmosispy
from osmosispy import utils, pytypes, Coin

In [14]:
# configure
mnemonic_key = "fat patch excite gold bubble large tunnel vote fine title hover junior advice cable ordinary column mass aunt trigger lucky hope animal abandon mansion"

## Prepare the account

to start localnet clone the [osmosis-labs/osmosis](https://github.com/osmosis-labs/osmosis) repo and run the following command

```bash
make localnet-build
make localnet-start
```

In [15]:
# # authorize in the localnet
# network = osmosispy.Network.mainnet()

# authorize in the mainnet
network = osmosispy.Network.mainnet()

trader = osmosispy.Sdk.authorize(key=mnemonic_key).with_network(network).with_config(pytypes.TxConfig())

# check the address
print(trader.address)

osmo1jggt8pcj2d8m9n62luytf8sdncj5uxfs3su2my


## Check balances of the account

In [16]:
from osmosis_proto.proto.cosmos.bank.v1beta1 import (
  query_pb2 as bank_query,
  query_pb2_grpc as bank_query_grpc,
)

In [17]:
stub_bank = bank_query_grpc.QueryStub(trader.query.chain_channel)
result = utils.deserialize(
    stub_bank.AllBalances(
        bank_query.QueryAllBalancesRequest(address=trader.address)
    )
)

print(result["balances"])


[]


## Query Pools

In [18]:
from osmosis_proto.proto.osmosis.gamm.v1beta1 import (
  query_pb2 as gamm_query,
  query_pb2_grpc as gamm_query_grpc,
)

In [19]:
stub_gamm = gamm_query_grpc.QueryStub(trader.query.chain_channel)
result = utils.deserialize(
    stub_gamm.Pools(
        gamm_query.QueryPoolsRequest()
    )
)

print(result["pools"])

[type_url: "/osmosis.gamm.v1beta1.Pool"
value: "\n?osmo1mw0ac6rwlp5r8wapwk3zs6g29h8fcscxqakdzw9emkne6c8wjp9q0t3v8t\020\001\032\025\n\0202000000000000000\022\0010\"\00324h**\n\013gamm/pool/1\022\0332337664426040038717658422912h\nU\nDibc/27394FB092D2ECCD56123C74F36E4C1F926001CEADA9CA97EA622B25F41E5EB2\022\r2337346178105\022\0175368709120000002*\n\027\n\005uosmo\022\01633605867545819\022\017536870912000000:\0201073741824000000"
, type_url: "/osmosis.gamm.v1beta1.Pool"
value: "\n?osmo1500hy75krs9e8t50aav6fahk8sxhajn9ctp40qwvvn8tcprkk6wszun4a5\020\002\032\025\n\0205000000000000000\022\0010\"\00324h**\n\013gamm/pool/2\022\0332657798302375781642667903942$\n\021\n\004uion\022\t642839378\022\0178589934592000002(\n\025\n\005uosmo\022\014151830330661\022\017214748364800000:\0201073741824000000"
, type_url: "/osmosis.gamm.v1beta1.Pool"
value: "\n?osmo1c9gj5nwxhuh2gz7wwg4r8e8tw8v7ggy9lh2hu7kkdgh0t450754qh9cpvd\020\003\032\025\n\0202000000000000000\022\0010\"\00324h*)\n\013gamm/pool/3\022\0328798744

## Query Locks

In [20]:
from osmosis_proto.proto.osmosis.lockup import (
    query_pb2 as lockup_query,
    query_pb2_grpc as lockup_query_grpc,
)

In [21]:
# Get account locks

stub_lockup = lockup_query_grpc.QueryStub(trader.query.chain_channel)
result = utils.deserialize(
    stub_lockup.AccountLockedDuration(
        # pass owner address string to query
        lockup_query.AccountLockedDurationRequest(owner=trader.address)
    )
)

print(result["locks"])

[]


In [22]:
# Lock by ID

stub_lockup = lockup_query_grpc.QueryStub(trader.query.chain_channel)
result = utils.deserialize(
    stub_lockup.LockedByID(
        # pass lock id to query
        lockup_query.LockedRequest(lock_id=123)
    )
)

print(result)

_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNKNOWN
	details = "lock with ID 123 does not exist: lockup not found"
	debug_error_string = "UNKNOWN:Error received from peer ipv4:137.184.114.151:9090 {created_time:"2023-04-05T18:58:32.416852344+01:00", grpc_status:2, grpc_message:"lock with ID 123 does not exist: lockup not found"}"
>

## Query Epochs

In [ ]:
from osmosis_proto.proto.osmosis.epochs import (
    query_pb2 as epochs_query,
    query_pb2_grpc as epochs_query_grpc,
)

In [ ]:
stub_epochs = epochs_query_grpc.QueryStub(trader.query.chain_channel)

In [ ]:
# Get all epochs
result = utils.deserialize(
    stub_epochs.EpochInfos(
        epochs_query.QueryEpochsInfoRequest()
    )
)

print(result)

In [ ]:
# Get current epoch
result = utils.deserialize(
    stub_epochs.CurrentEpoch(
        epochs_query.QueryCurrentEpochRequest(identifier="day")
    )
)

print(result)

## Get pools

In [ ]:
from osmosis_proto.proto.osmosis.gamm.v1beta1 import (
    query_pb2 as gamm_query,
    query_pb2_grpc as gamm_query_grpc,
)

In [ ]:
stub_gamm = gamm_query_grpc.QueryStub(trader.query.chain_channel)
pools_resp = utils.deserialize(
    stub_gamm.Pools(
        gamm_query.QueryPoolsRequest()
    )
)

# pools need to be deserialized manually as they are Any types
pools = [utils.deserialize_pool(pool) for pool in pools_resp["pools"]]

print("pools", pools)

In [ ]:
# Get specific pool by ID
stub_gamm = gamm_query_grpc.QueryStub(trader.query.chain_channel)
pool_raw_data = utils.deserialize(
    stub_gamm.Pool(
        # checking this pool https://app.osmosis.zone/pool/678
        gamm_query.QueryPoolRequest(pool_id=678)
    )
)

pool = utils.deserialize_pool(pool_raw_data["pool"])

print(pool)

In [ ]:
# Get spot price
price_data = utils.deserialize(
    stub_gamm.SpotPrice(
        gamm_query.QuerySpotPriceRequest(
            pool_id=pool["id"],
            base_asset_denom=pool["pool_assets"][0]["token"]["denom"],
            quote_asset_denom=pool["pool_assets"][1]["token"]["denom"],
        )
    )
)

print(price_data["spot_price"])